# Capstone project - week 3 - part 2
Getting location Dataframe

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

In [2]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Libraries imported.")

Libraries imported.


In [3]:
import folium # map rendering library
print("Libraries imported.")

Libraries imported.


In [4]:
#getting the rquired html page
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
#Creating soup objeect
soup = BeautifulSoup(data, 'html.parser')

In [6]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:
#getting the dataframe
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#dropping all rows with non-assigned borough
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
#getting comma separated value for 2 or more neighbourhoods
df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pcn

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
df_pcn.shape

(103, 3)

In [12]:
#importing csv of geospatial coordinates
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
coordinates.rename(columns={"Postal Code": "Postcode"}, inplace=True)
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#merging the both datasets
 df_pos = df_pcn.merge(coordinates, on=['Postcode'], how='left')
df_pos.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
column_names = ["Postcode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)
test_df

,Postcode,Borough,Neighborhood,Latitude,Longitude


In [39]:
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for postcode in test_list:
    test_df = test_df.append(df_pos[df_pos["Postcode"]==postcode], ignore_index=True)
    
test_df.isnull().sum()

Postcode          0
Borough           0
Neighborhood     12
Latitude          0
Longitude         0
Neighbourhood     0
dtype: int64

In [40]:
column_names = ["Postcode", "Borough", "Neighborhood2", "Latitude", "Longitude","Neighborhood"]
test_df.columns= column_names

In [41]:
test_df

,Postcode,Borough,Neighborhood2,Latitude,Longitude,Neighborhood
0,M5G,Downtown Toronto,NaN,43.657952,-79.387383,Central Bay Street
1,M2H,North York,NaN,43.803762,-79.363452,Hillcrest Village
2,M4B,East York,NaN,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,NaN,43.744734,-79.239476,Scarborough Village
4,M4G,East York,NaN,43.709060,-79.363452,Leaside
5,M4M,East Toronto,NaN,43.659526,-79.340923,Studio District
6,M1R,Scarborough,NaN,43.750072,-79.295849,"Wexford, Maryvale"
7,M9V,Etobicoke,NaN,43.739416,-79.588437,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,NaN,43.756303,-79.565963,Humber Summit
9,M5V,Downtown Toronto,NaN,43.628947,-79.394420,"CN Tower, King and Spadina, Railway Lands, Har..."


In [42]:
test_df.drop("Neighborhood2",axis=1,inplace=True)

In [43]:
test_df

,Postcode,Borough,Latitude,Longitude,Neighborhood
0,M5G,Downtown Toronto,43.657952,-79.387383,Central Bay Street
1,M2H,North York,43.803762,-79.363452,Hillcrest Village
2,M4B,East York,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,43.744734,-79.239476,Scarborough Village
4,M4G,East York,43.709060,-79.363452,Leaside
5,M4M,East Toronto,43.659526,-79.340923,Studio District
6,M1R,Scarborough,43.750072,-79.295849,"Wexford, Maryvale"
7,M9V,Etobicoke,43.739416,-79.588437,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,43.756303,-79.565963,Humber Summit
9,M5V,Downtown Toronto,43.628947,-79.394420,"CN Tower, King and Spadina, Railway Lands, Har..."


In [44]:
column_names = ["Postcode", "Borough", "Neighborhood","Latitude", "Longitude"]
test_df= test_df[column_names]
test_df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
